In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings("ignore")

In [11]:
import splitfolders
splitfolders.ratio("Data", output="output",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 3784 files [00:05, 729.03 files/s]


In [5]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = 20

In [6]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size=(224,224),
                                                 batch_size=16)

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size=(224,224),
                                            batch_size=16)

Found 3018 images belonging to 20 classes.
Found 766 images belonging to 20 classes.


In [7]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(key, activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=3e-4, momentum=0.9),
        #optimizer = tf.keras.optimizers.Nadam(learning_rate=3e-4, beta_1=0.9, 
                                            #  beta_2=0.999, epsilon=1e-07, name="Nadam"),
        # Set loss to binary crossentropy
        #loss = tf.keras.losses.sparse_categorical_crossentropy,
        loss = tf.keras.losses.CategoricalCrossentropy(),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=2,
                                                            verbose=2,
                                                            factor=0.5,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

In [9]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    history = model.fit(training_set, epochs= epochs , verbose = 2, validation_data = test_set,
                       callbacks=[early_stopping, reduce_lr , learning_rate_reduction])
    
    

    model.save_weights('layer22-wskindiseases1.h5')
    model.save('layer22-skindiseases1.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [10]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('Layer22-wskindiseases1.h5')
    mod.evaluate(test_set)
    
    #prob = mod.predict(test_set)
    #predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    #print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return predIdxs, prob, mod 

In [20]:
if __name__ == "__main__":
    
    train_history = train() 
    predIdxs,prob,  model = test()
    
    show_train_history(train_history, 'sparse_categorical_accuracy')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

Epoch 1/500


C:\Users\laptop\anaconda3\envs\gpu_test\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 - 97s - loss: 2.4694 - accuracy: 0.2568 - val_loss: 1.8605 - val_accuracy: 0.4491
Epoch 2/500
189/189 - 89s - loss: 1.7030 - accuracy: 0.4877 - val_loss: 1.5761 - val_accuracy: 0.5157
Epoch 3/500
189/189 - 91s - loss: 1.2719 - accuracy: 0.6107 - val_loss: 1.2778 - val_accuracy: 0.6253
Epoch 4/500
189/189 - 90s - loss: 0.9162 - accuracy: 0.7256 - val_loss: 1.3603 - val_accuracy: 0.5953
Epoch 5/500
189/189 - 90s - loss: 0.7136 - accuracy: 0.7866 - val_loss: 0.9528 - val_accuracy: 0.7115
Epoch 6/500
189/189 - 90s - loss: 0.5424 - accuracy: 0.8290 - val_loss: 1.0408 - val_accuracy: 0.7219
Epoch 7/500
189/189 - 90s - loss: 0.4439 - accuracy: 0.8625 - val_loss: 1.0243 - val_accuracy: 0.7219
Epoch 8/500
189/189 - 90s - loss: 0.3424 - accuracy: 0.8989 - val_loss: 1.0185 - val_accuracy: 0.7428
Epoch 9/500
189/189 - 90s - loss: 0.2123 - accuracy: 0.9404 - val_loss: 0.7371 - val_accuracy: 0.8342
Epoch 10/500
189/189 - 90s - loss: 0.1249 - accuracy: 0.9655 - val_loss: 0.7606 - val_accuracy

C:\Users\laptop\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Model saved.
Training Time: 2255.4112713336945


Testing:
48/48 [==============================] - 6s 102ms/step - loss: 0.7718 - accuracy: 0.8499




NameError: name 'predIdxs' is not defined

In [32]:
input_path = 'Data/'

In [33]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = train_data.class_names

In [34]:
def read_directory():
    data_filenames = []
    data_labels = []
    for i in range (len(key)):
        for filename in os.listdir(input_path + key[i]):
                data_filenames.append(input_path +key[i]+'/' + filename)
                data_labels.append(i)
  
    return  data_filenames, data_labels

In [35]:
def build_test_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [36]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [37]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [38]:
def data_Preprocessing( test_dataset):
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return  test_dataset

In [39]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(len(key), activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=0.0001, momentum=0.9),
        # Set loss to binary crossentropy
        loss = tf.keras.losses.sparse_categorical_crossentropy,

        # Set metrics to accuracy
        metrics = ['accuracy']
    )


    return model

In [40]:
def test(test_labels):
    test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('layer22-wskindiseases1.h5')
    mod.evaluate(test_dataset)
    
    prob = mod.predict(test_dataset)
    predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    print(classification_report(test_labels, predIdxs,target_names = key, digits=5))
    return test_labels, predIdxs, prob, mod

In [41]:
if __name__ == "__main__":
    
    test_filenames, test_labels = read_directory()
    test_tfrecord = 'XRay_test.tfrecords'
    build_test_tfrecord(test_filenames, test_labels)

    test_dataset = get_test_dataset(test_tfrecord)


    test_dataset = data_Preprocessing(test_dataset) 
    test_labels, predIdxs,prob, Model = test(test_labels)
    


Testing:
237/237 [==============================] - 25s 101ms/step - loss: 0.1660 - accuracy: 0.9641


                                        precision    recall  f1-score   support

                          Atrophoderma    0.96907   0.94000   0.95431       100
                      Calcinosis Cutis    0.97183   0.96729   0.96956       214
                  Chilblains Perniosis    0.93506   0.93506   0.93506        77
                       Dermatomyositis    0.97933   0.97179   0.97555       390
           Discoid lupus erythematosus    0.94302   0.95665   0.94978       346
                 Dupuytren Contracture    0.98148   0.99065   0.98605       107
                       Erythromelalgia    0.95604   0.95604   0.95604        91
                           Lupus Acute    1.00000   0.93617   0.96703        47
               Lupus Chronic Cutaneous    0.96454   0.97143   0.96797       140
                        Lupus Neonatal    0.98477   0.98980   0.98728       196
                

In [42]:
def dermai_accuracy(prob, prob_Desicon):
    dermai = []
    for i in range(len(prob)):
        catogery =[]
        for j in range(len(prob[i])):
            if prob[i][j] > prob_Desicon:
                catogery.append(j)
        dermai.append(catogery)
    Positive = []
    for i in range(len(dermai)):
        if test_labels[i] in dermai[i]:
            Positive.append(1)
    accuracy = (len(Positive)/len(test_labels))*100  
    return  accuracy,dermai

In [43]:
accuracy,dermai = dermai_accuracy(prob, 0.10)

In [44]:
accuracy

97.6215644820296